In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
from bingX import BingX
import mplfinance as mpl
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
import pandas_ta as ta

TRADE_INTERVALS = ["1M","1d","4h","1h","30m","15m","5m","3m","1m"]

# Exchange
API_KEY = os.environ.get("BINGX_API_KEY","")
SECRET_KEY = os.environ.get("BINGX_SECRET_KEY","")
bingx_client = BingX(API_KEY, SECRET_KEY)

def get_data(symbol, interval, start, end):
    start_time_ms = int(start.timestamp() * 1000)
    end_time_ms = int(end.timestamp() * 1000)
    data = bingx_client.perpetual_v2.market.get_k_line_data(symbol=symbol,interval=interval,start_time=start_time_ms,end_time=end_time_ms)
    df = pd.DataFrame(data)
    df.set_index('time', inplace=True)
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df


def get_levels(date):
    ratios = [-0.618, 0.618, 1.618] # SL - Entry - TP
    series_ = df.loc[date:][1:2].squeeze()
    diff = series_.high - series_.low
    levels = [series_.close + i * diff for i in ratios]
    return levels

def get_median_values(lf):
    n = 5
    lf['median'] = np.abs(lf['high'] + lf['low']) / 2
    lf['min'] = lf.iloc[argrelextrema(lf['median'].values, np.less_equal,
                                      order=n)[0]]['median']
    lf['max'] = lf.iloc[argrelextrema(lf['median'].values, np.greater_equal,
                                      order=n)[0]]['median']
    return lf

# Date intervals to fetch data
date = datetime(2022,1,1,0,0,0)
date_end = datetime(2022,1,5,0,0,0)
df = get_data("BTC-USDT", "1h", date, date_end)
lf = get_median_values(df)

#lf = df.drop(columns=['volume','open','close'])
#pdf = lf.drop(columns=['high','low'])
plt.scatter(lf.index,lf['min'],c='r')
plt.scatter(lf.index,lf['max'],c='g')
plt.plot(lf.index,lf['median'])
plt.show()

In [ ]:
# Fibo back testing (WIP) - Still checking if this make sense.
buys,sells = [],[]
trade_dates = []
in_position = False
df['price'] = df.open.shift(-1)
days_range = np.unique(df.index.date)
for date in days_range:
    for index, row in df[date:][2:].iterrows():
        if not in_position:
            sl,entry,tp = get_levels(date)
            if row.close >= entry:
                df.at[index, 'buy'] = row.price
                buys.append(row.price)
                trade_dates.append(date)
                in_position = True
        if in_position:
            if row.close >= tp or row.close <= sl:
                df.at[index, 'sell'] = row.price
                sells.append(row.price)
                in_position = False

trades = pd.DataFrame([buys,sells])
trades.columns = trade_dates
trades.index = ['Buy','Sell']
trades = trades.T


In [ ]:
def fibonacci_levels(max_val, min_val):
    ratios = [-0.618, 0.618, 1.618] # SL - Entry - TP
    diff = max_val - min_val
    levels = [max_val - i * diff for i in ratios]
    return levels

In [ ]:
filtered_df = lf[lf['min'].notnull() | lf['max'].notnull()]
filtered_df

In [ ]:
idx = 0
min_max_vals = []
min_max_type = {0:[],1:[]}
print(filtered_df)
while(idx < len(filtered_df)):
    row = filtered_df.iloc[idx]
    if not pd.isna(row['min']):
        print("Min: " + str(row['min']))
        min_max_vals.append(row['min'])
        min_max_type[0].append(row['min'])
    if not pd.isna(row['max']):
        print("Max: " + str(row['max']))
        min_max_vals.append(row['max'])
        min_max_type[1].append(row['max'])
    idx = idx + 1